In [1]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
import numpy as np
sys.path.append("../")
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
from binary_ops import *
%matplotlib qt

In [ ]:
np.max(ims[0][10])

In [2]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../../data/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.hdf", "r+")

    images = np.array(file["/data"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val



ball_vids = read_many_hdf5("balls_5k")
ball_vids = np.transpose(ball_vids,(0,3,1,2))
ball_vids = np.reshape(ball_vids,(5000,30,32,32,1))
ims = ball_vids
print(np.shape(ball_vids))
validate = ims

#validate = validate / 255
#ims2 = ims2 /255
#ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 0.3, random_state = 42)

print(np.shape(MX_test))
print(np.shape(MX_train))


reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.5,
                              patience=20, min_lr=1e-7)
early_stopping = EarlyStopping(patience=90,verbose=1,restore_best_weights=True)   


def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.5*ssim_loss
  mse_loss = 0.5*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))

(5000, 30, 32, 32, 1)
(1500, 30, 32, 32, 1)
(3500, 30, 32, 32, 1)


In [ ]:
show_video(MX_test, My_test, 100)

In [3]:
""" VIDEO FUNCTIONS FOR CHECKING POST TRAINING"""

def get_mask(model,l=0, save = False, filename = "mask"):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")
    
    if save:
        b = np.reshape(b,(32,32))
        np.save(filename,b)
                            
def show_video(y_pred,y_true,num,save=False,filename='test.png'):
    yp = np.reshape(y_pred,(-1,30,32,32))
    yt = np.reshape(y_true,(-1,30,32,32))
    split = np.zeros((5,32,32))
    yp_tensor = tf.convert_to_tensor(y_pred)
    yt_tensor = tf.convert_to_tensor(y_true)
    ssim = np.mean(tf.image.ssim(yt_tensor[num],yp_tensor[num],1))
    mse = np.mean(mse_loss(yt_tensor[num],yp_tensor[num]))
    fig,ax = plt.subplots(nrows=5,ncols=13,figsize=(21,10),sharex=True,sharey=True)
    fig.suptitle(f'Movie: {num} MSE: {mse:.3} SSIM: {ssim:.3}')
    for row in range(5):
        for col in range(13):
            if col < 6:
                ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            elif col == 6:
                pass
            else:
                ax[row,col].imshow(yt[num][5*row + (col %7)],cmap="gray")
    if save:
        fig.savefig(filename)

def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")

In [6]:
""" FORWARD MODEL """

forward_model = Sequential()
forward_model.add(Input(shape=(30,32,32,1)))
forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                   data_format='channels_last',
                   H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                   padding='same', use_bias=use_bias, name='bin_conv_1')))
forward_model.add(Reshape((30,32,32)))
forward_model.add(Lambda(streak,output_shape=streak_output_shape))
forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
forward_model.add(Flatten())
forward_model.add(Dense(30720, activation = 'relu'))
forward_model.add(Reshape((30,32,32,1)))
forward_model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = [ssim_loss,mse_loss])
forward_model.summary()
forward_history = forward_model.fit(MX_train, My_train,
      batch_size = 32,epochs= 150,
    verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr, early_stopping])


    

(None, 30, 62, 32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 32, 32, 1)     1024      
_________________________________________________________________
reshape (Reshape)            (None, 30, 32, 32)        0         
_________________________________________________________________
lambda (Lambda)              (None, 30, 62, 32)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 62, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1984)              0         
_________________________________________________________________
dense (Dense)                (None, 30720)             60979200  
_________________________________________________________________
reshape_1 (Reshape)          (None, 3

Epoch 47/150
3500/3500 - 18s - loss: 0.1908 - ssim_loss: 0.2899 - mse_loss: 0.0917 - val_loss: 0.1941 - val_ssim_loss: 0.2955 - val_mse_loss: 0.0926
Epoch 48/150
3500/3500 - 18s - loss: 0.1906 - ssim_loss: 0.2897 - mse_loss: 0.0916 - val_loss: 0.1940 - val_ssim_loss: 0.2954 - val_mse_loss: 0.0925
Epoch 49/150
3500/3500 - 18s - loss: 0.1905 - ssim_loss: 0.2894 - mse_loss: 0.0916 - val_loss: 0.1939 - val_ssim_loss: 0.2951 - val_mse_loss: 0.0927
Epoch 50/150
3500/3500 - 18s - loss: 0.1904 - ssim_loss: 0.2892 - mse_loss: 0.0916 - val_loss: 0.1939 - val_ssim_loss: 0.2951 - val_mse_loss: 0.0927
Epoch 51/150
3500/3500 - 18s - loss: 0.1903 - ssim_loss: 0.2890 - mse_loss: 0.0916 - val_loss: 0.1936 - val_ssim_loss: 0.2951 - val_mse_loss: 0.0922
Epoch 52/150
3500/3500 - 18s - loss: 0.1901 - ssim_loss: 0.2887 - mse_loss: 0.0915 - val_loss: 0.1934 - val_ssim_loss: 0.2944 - val_mse_loss: 0.0924
Epoch 53/150
3500/3500 - 18s - loss: 0.1900 - ssim_loss: 0.2885 - mse_loss: 0.0915 - val_loss: 0.1935 - va

KeyboardInterrupt: 

In [ ]:
y_pred_forward = forward_model.predict(MX_test)
y_pred_forward 
show_video(y_pred_forward,MX_test,300)

In [24]:
CUPNET_X_train = forward_model.predict(MX_train)
CUPNET_X_test = forward_model.predict(MX_test)

In [ ]:
forward_model.save_weights("../../data/model_stuff/ball_videos_forward_weights_5_2.h5")
binary_weights = forward_model.layers[0].get_weights()
inverse_weights = forward_model.layers[5].get_weights()
get_mask(forward_model)

In [ ]:
show_video(CUPNET_X_train,My_train,100)

In [4]:
forward_model = Sequential()
forward_model.add(Input(shape=(30,32,32,1)))
forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                   data_format='channels_last',
                   H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                   padding='same', use_bias=use_bias, name='bin_conv_1')))
forward_model.add(Reshape((30,32,32)))
forward_model.add(Lambda(streak,output_shape=streak_output_shape))
forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
forward_model.add(Flatten())
forward_model.add(Dense(30720, activation = 'relu'))
forward_model.add(Reshape((30,32,32,1)))
forward_model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = [ssim_loss,mse_loss])
forward_model.summary()

(None, 30, 62, 32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 32, 32, 1)     1024      
_________________________________________________________________
reshape (Reshape)            (None, 30, 32, 32)        0         
_________________________________________________________________
lambda (Lambda)              (None, 30, 62, 32)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 62, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1984)              0         
_________________________________________________________________
dense (Dense)                (None, 30720)             60979200  
_________________________________________________________________
reshape_1 (Reshape)          (None, 3

In [6]:
forward_model.load_weights("../../data/model_stuff/ball_videos_forward_weights_5_2.h5")
b = forward_model.layers[0].get_weights()
b = binarize(tf.convert_to_tensor(b))

In [ ]:
""" 
UNET MODEL
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""
inputs = Input(shape=(30,32,32,1))
bin_conv1 = TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1',trainable=False))(inputs)
resh1 = Reshape((30,32,32))(bin_conv1)
s = Lambda(streak, output_shape = streak_output_shape)(resh1)
i = Lambda(integrate_ims, output_shape = integrate_ims_output_shape) (s)
f = Flatten()(i)
dense1 = Dense(30720, activation = 'relu',trainable=False)(f)
resh2 = Reshape((30,32,32,1))(dense1)
c1 = TimeDistributed(Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (resh2)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
p1 = TimeDistributed(Conv2D(1, (3, 3),strides=2, activation='relu', kernel_initializer='he_normal', padding='same'))(c1)
b1 = BatchNormalization() (p1)

c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(b1)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
p2 = TimeDistributed(Conv2D(1, (3, 3),strides=2, activation='relu', kernel_initializer='he_normal', padding='same'))(c2)
b2 = BatchNormalization() (p2)

c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(b2)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
p3 = TimeDistributed(Conv2D(1, (3, 3),strides=2, activation='relu', kernel_initializer='he_normal', padding='same'))(c3)
b3 = BatchNormalization() (p3)
    
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(b3)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
p4 = TimeDistributed(Conv2D(1, (3, 3),strides=2, activation='relu', kernel_initializer='he_normal', padding='same'))(c4)
b4 = BatchNormalization() (p4)

c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (b4)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)

u6 = TimeDistributed(Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)

u7 = TimeDistributed(Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
    
u8 = TimeDistributed(Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)

u9 = TimeDistributed(Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET.compile(optimizer = Nadam(), loss = 'mse',callbacks=[reduce_lr, early_stopping])
CUPNET.summary()

In [25]:
""" 
UNET MODEL
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""
inputs = Input(shape=(30,32,32,1))
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (inputs)
c1 = BatchNormalization() (c1)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
c1 = BatchNormalization() (c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = BatchNormalization() (c2)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
c2 = BatchNormalization() (c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = BatchNormalization() (c3)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
c3 = BatchNormalization() (c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)

c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = BatchNormalization() (c4)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
c4 = BatchNormalization() (c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = BatchNormalization() (c5)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)
c5 = BatchNormalization() (c5)

u6 = TimeDistributed(Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = BatchNormalization() (c6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)
c6 = BatchNormalization() (c6)

u7 = TimeDistributed(Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = BatchNormalization() (c7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
c7 = BatchNormalization() (c7)
    
u8 = TimeDistributed(Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = BatchNormalization() (c8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)
c8 = BatchNormalization() (c8)

u9 = TimeDistributed(Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = BatchNormalization() (c9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
c9 = BatchNormalization() (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET2 = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET2.compile(optimizer = Nadam(), loss = custom_loss, metrics = [ssim_loss,mse_loss],callbacks=[reduce_lr, early_stopping])
CUPNET2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 32, 32,  0                                            
__________________________________________________________________________________________________
time_distributed_28 (TimeDistri (None, 30, 32, 32, 3 320         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 30, 32, 32, 3 128         time_distributed_28[0][0]        
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 30, 32, 32, 3 0           batch_normalization_18[0][0]     
____________________________________________________________________________________________

In [26]:
CUPNET2_history = CUPNET2.fit(CUPNET_X_train, My_train,
          batch_size = 32,epochs= 300,
          verbose=2,validation_data=(CUPNET_X_test,My_test),callbacks=[reduce_lr,early_stopping])

Train on 3500 samples, validate on 1500 samples
Epoch 1/300
3500/3500 - 157s - loss: 0.1899 - ssim_loss: 0.3077 - mse_loss: 0.0719 - val_loss: 0.1898 - val_ssim_loss: 0.2731 - val_mse_loss: 0.1065
Epoch 2/300
3500/3500 - 132s - loss: 0.0811 - ssim_loss: 0.1316 - mse_loss: 0.0306 - val_loss: 0.1160 - val_ssim_loss: 0.1887 - val_mse_loss: 0.0432
Epoch 3/300
3500/3500 - 131s - loss: 0.0604 - ssim_loss: 0.1009 - mse_loss: 0.0199 - val_loss: 0.0823 - val_ssim_loss: 0.1355 - val_mse_loss: 0.0291
Epoch 4/300
3500/3500 - 131s - loss: 0.0540 - ssim_loss: 0.0910 - mse_loss: 0.0171 - val_loss: 0.0676 - val_ssim_loss: 0.1129 - val_mse_loss: 0.0224
Epoch 5/300
3500/3500 - 131s - loss: 0.0502 - ssim_loss: 0.0849 - mse_loss: 0.0156 - val_loss: 0.0616 - val_ssim_loss: 0.1035 - val_mse_loss: 0.0197
Epoch 6/300
3500/3500 - 131s - loss: 0.0479 - ssim_loss: 0.0811 - mse_loss: 0.0147 - val_loss: 0.0624 - val_ssim_loss: 0.1050 - val_mse_loss: 0.0198
Epoch 7/300
3500/3500 - 131s - loss: 0.0464 - ssim_loss: 0

Epoch 55/300
3500/3500 - 131s - loss: 0.0246 - ssim_loss: 0.0417 - mse_loss: 0.0075 - val_loss: 0.0577 - val_ssim_loss: 0.0974 - val_mse_loss: 0.0180
Epoch 56/300
3500/3500 - 131s - loss: 0.0244 - ssim_loss: 0.0413 - mse_loss: 0.0074 - val_loss: 0.0572 - val_ssim_loss: 0.0964 - val_mse_loss: 0.0179
Epoch 57/300
3500/3500 - 131s - loss: 0.0242 - ssim_loss: 0.0411 - mse_loss: 0.0074 - val_loss: 0.0574 - val_ssim_loss: 0.0969 - val_mse_loss: 0.0179
Epoch 58/300
3500/3500 - 131s - loss: 0.0241 - ssim_loss: 0.0408 - mse_loss: 0.0073 - val_loss: 0.0578 - val_ssim_loss: 0.0976 - val_mse_loss: 0.0181
Epoch 59/300
3500/3500 - 131s - loss: 0.0239 - ssim_loss: 0.0405 - mse_loss: 0.0073 - val_loss: 0.0577 - val_ssim_loss: 0.0973 - val_mse_loss: 0.0181
Epoch 60/300
3500/3500 - 131s - loss: 0.0239 - ssim_loss: 0.0404 - mse_loss: 0.0073 - val_loss: 0.0577 - val_ssim_loss: 0.0974 - val_mse_loss: 0.0181
Epoch 61/300
3500/3500 - 131s - loss: 0.0237 - ssim_loss: 0.0401 - mse_loss: 0.0072 - val_loss: 0.05

Epoch 109/300
3500/3500 - 131s - loss: 0.0194 - ssim_loss: 0.0327 - mse_loss: 0.0060 - val_loss: 0.0581 - val_ssim_loss: 0.0979 - val_mse_loss: 0.0184
Epoch 110/300

Epoch 00110: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
3500/3500 - 131s - loss: 0.0193 - ssim_loss: 0.0326 - mse_loss: 0.0060 - val_loss: 0.0579 - val_ssim_loss: 0.0976 - val_mse_loss: 0.0183
Epoch 111/300
3500/3500 - 131s - loss: 0.0192 - ssim_loss: 0.0324 - mse_loss: 0.0060 - val_loss: 0.0581 - val_ssim_loss: 0.0978 - val_mse_loss: 0.0183
Epoch 112/300
3500/3500 - 131s - loss: 0.0192 - ssim_loss: 0.0324 - mse_loss: 0.0060 - val_loss: 0.0581 - val_ssim_loss: 0.0979 - val_mse_loss: 0.0184
Epoch 113/300
3500/3500 - 131s - loss: 0.0191 - ssim_loss: 0.0323 - mse_loss: 0.0060 - val_loss: 0.0580 - val_ssim_loss: 0.0977 - val_mse_loss: 0.0183
Epoch 114/300
3500/3500 - 131s - loss: 0.0190 - ssim_loss: 0.0321 - mse_loss: 0.0060 - val_loss: 0.0582 - val_ssim_loss: 0.0979 - val_mse_loss: 0.0184
Epoch 115/300


In [ ]:
(np.square(np.zeros((32,32))-np.reshape(My_train[10][10],(32,32))).mean(axis=None))

In [28]:
y_train_pred = CUPNET2.predict(CUPNET_X_train)
y_test_pred = CUPNET2.predict(CUPNET_X_test)
np.shape(y_train_pred[10][20])

(32, 32, 1)

In [29]:
np.save('ball_test_predictions',y_test_pred)
np.save('ball_train_predictions',y_train_pred)
np.save('ball_training_ims',My_train)
np.save('ball_test_ims',My_test)

In [30]:
show_video(y_train_pred,MX_train,200)

In [31]:
show_video(y_test_pred,My_test,200)

In [ ]:
CUPNET.load_weights("../../data/model_stuff/cupnet_jelly_weights_4_23.h5")

In [ ]:
b = CUPNET.layers[1].get_weights()
b = binarize(tf.convert_to_tensor(b))
b = np.reshape(b,(32,32))
np.save('cupnet_jelly_mask',b)

In [ ]:
CUPNET.layers[1].set_weights(binary_weights)
CUPNET.layers[6].set_weights(inverse_weights)
CUPNET.summary()

In [ ]:
""" CUPNET TRAINING """
CUPNET_history = CUPNET.fit(MX_train, My_train,
          batch_size = 32,epochs= 100,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr,early_stopping])

In [27]:
CUPNET2.save_weights("../../data/model_stuff/cupnet2_ball_video_weights_5_3.h5")

In [ ]:
print(CUPNET_history.history.keys())

In [ ]:
# summarize history for loss
plt.plot(CUPNET_history.history['loss'])
plt.plot(CUPNET_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(forward_history.history['loss'])
plt.plot(forward_history.history['val_loss'])
plt.title('forward_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_train_pred = CUPNET.predict(MX_train)
y_test_pred = CUPNET.predict(MX_test)


In [21]:
show_video(y_test_pred,MX_test,200,True,'test_balls_200.png')

In [22]:
show_video(y_train_pred,MX_train,200,True,'train_balls_200.png')

In [ ]:
def store_many_hdf5(images,name):

    num_images = len(images)
    try:
        os.mkdir("../../data/hdf5")
    except: 
        pass

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{name}_vids.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", data=images
    )    
    
    file.close()

In [ ]:
np.save("../../data/hdf5/u_net_jelly_test_ims",y_test_pred)

In [ ]:
store_many_hdf5(y_test_pred,"u_net_predict")

In [ ]:
import scipy.io as sio
twist_jelly = sio.loadmat('../../data/hdf5/jelly_resized.mat')
jelly = np.asarray(twist_jelly['sample'])
jelly = np.transpose(jelly,(2,0,1))
jelly = np.reshape(jelly[30:60], (1,30,32,32,1))
print(np.shape(jelly))

In [ ]:
j_pred = CUPNET.predict(jelly)